In [1]:
import pandas as pd
import numpy as np
import allel
import json

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
refgene = pd.read_csv('data/refseq-20200109.txt', sep='\t')
refgene['length'] = refgene['txEnd'] - refgene['txStart']
refgene = refgene.sort_values('length', ascending=False)
refgene = refgene.drop_duplicates('name2', keep='first')

In [4]:
refgene.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames,length
21052,26,NM_014141.5,chr7,+,145813452,148118090,145813968,148112708,24,"145813452,146471362,146536802,146740998,146805...","145814065,146471473,146536996,146741146,146805...",0,CNTNAP2,cmpl,cmpl,"0,1,1,0,1,1,0,0,1,1,2,1,1,1,2,1,1,1,1,1,0,1,1,1,",2304638
25593,1,NM_002839.3,chr9,-,8314245,10612723,8317873,8733843,46,"8314245,8319830,8331581,8338921,8340342,834108...","8317942,8319966,8331736,8339047,8340469,834126...",0,PTPRD,cmpl,cmpl,"0,2,0,0,2,0,2,0,0,1,0,1,2,0,0,2,0,1,1,0,0,1,1,...",2298478
27890,12,NM_000109.3,chrX,-,31137344,33357726,31140035,33357382,79,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",0,DMD,cmpl,cmpl,"0,1,1,0,2,2,2,2,2,0,2,0,1,2,1,1,2,1,0,0,1,0,2,...",2220382
33217,2,NM_001142699.1,chr11,-,83166054,85338314,83170860,85309740,28,"83166054,83173044,83177750,83180243,83182668,8...","83170967,83173136,83177860,83180416,83182770,8...",0,DLG2,cmpl,cmpl,"1,2,0,1,1,1,0,2,2,1,0,2,2,2,1,0,1,2,0,0,0,0,0,...",2172260
23572,9,NM_033225.5,chr8,-,2792874,4852328,2796106,4851938,70,"2792874,2799993,2806820,2807752,2808635,281174...","2796266,2800126,2806908,2807865,2808797,281179...",0,CSMD1,cmpl,cmpl,"2,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",2059454


In [5]:
gene_info = pd.read_csv('data/Homo_sapiens.gene_info.gz', sep='\t')

In [6]:
gene_info.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20191220,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20200113,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20191221,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171428,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20200113,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156006,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20200121,-


In [7]:
refgene_info = refgene.merge(gene_info, left_on='name2', right_on='Symbol')

In [8]:
refgene_info.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames,length,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,26,NM_014141.5,chr7,+,145813452,148118090,145813968,148112708,24,"145813452,146471362,146536802,146740998,146805...","145814065,146471473,146536996,146741146,146805...",0,CNTNAP2,cmpl,cmpl,"0,1,1,0,1,1,0,0,1,1,2,1,1,1,2,1,1,1,1,1,0,1,1,1,",2304638,9606,26047,CNTNAP2,-,AUTS15|CASPR2|CDFE|NRXN4|PTHSL1,MIM:604569|HGNC:HGNC:13830|Ensembl:ENSG0000017...,7,7q35-q36.1,contactin associated protein 2,protein-coding,CNTNAP2,contactin associated protein 2,O,contactin-associated protein-like 2|cell recog...,20200105,-
1,1,NM_002839.3,chr9,-,8314245,10612723,8317873,8733843,46,"8314245,8319830,8331581,8338921,8340342,834108...","8317942,8319966,8331736,8339047,8340469,834126...",0,PTPRD,cmpl,cmpl,"0,2,0,0,2,0,2,0,0,1,0,1,2,0,0,2,0,1,1,0,0,1,1,...",2298478,9606,5789,PTPRD,-,HPTP|HPTPD|HPTPDELTA|PTPD|R-PTP-delta|RPTPDELTA,MIM:601598|HGNC:HGNC:9668|Ensembl:ENSG00000153707,9,9p24.1-p23,protein tyrosine phosphatase receptor type D,protein-coding,PTPRD,protein tyrosine phosphatase receptor type D,O,receptor-type tyrosine-protein phosphatase del...,20200128,-
2,12,NM_000109.3,chrX,-,31137344,33357726,31140035,33357382,79,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",0,DMD,cmpl,cmpl,"0,1,1,0,2,2,2,2,2,0,2,0,1,2,1,1,2,1,0,0,1,0,2,...",2220382,9606,1756,DMD,-,BMD|CMD3B|DXS142|DXS164|DXS206|DXS230|DXS239|D...,MIM:300377|HGNC:HGNC:2928|Ensembl:ENSG00000198947,X,Xp21.2-p21.1,dystrophin,protein-coding,DMD,dystrophin,O,dystrophin,20200105,-
3,2,NM_001142699.1,chr11,-,83166054,85338314,83170860,85309740,28,"83166054,83173044,83177750,83180243,83182668,8...","83170967,83173136,83177860,83180416,83182770,8...",0,DLG2,cmpl,cmpl,"1,2,0,1,1,1,0,2,2,1,0,2,2,2,1,0,1,2,0,0,0,0,0,...",2172260,9606,1740,DLG2,-,PPP1R58|PSD-93|PSD93|chapsyn-110,MIM:603583|HGNC:HGNC:2901|Ensembl:ENSG00000150672,11,11q14.1,discs large MAGUK scaffold protein 2,protein-coding,DLG2,discs large MAGUK scaffold protein 2,O,disks large homolog 2|DLG2 protein isoform|cha...,20200124,-
4,9,NM_033225.5,chr8,-,2792874,4852328,2796106,4851938,70,"2792874,2799993,2806820,2807752,2808635,281174...","2796266,2800126,2806908,2807865,2808797,281179...",0,CSMD1,cmpl,cmpl,"2,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",2059454,9606,64478,CSMD1,-,PPP1R24,MIM:608397|HGNC:HGNC:14026|Ensembl:ENSG0000018...,8,8p23.2,CUB and Sushi multiple domains 1,protein-coding,CSMD1,CUB and Sushi multiple domains 1,O,CUB and sushi domain-containing protein 1|prot...,20200127,-


In [9]:
gene = refgene_info.loc[
    refgene_info['type_of_gene'] == 'protein-coding',
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [10]:
gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
22981,chr1,69090,70008,79501,OR4F5,NM_001005484.1,+
22876,chr1,367658,368597,729759,OR4F29,NM_001005221.2,+
22845,chr1,621095,622034,81399,OR4F16,NM_001005277.1,-
12034,chr1,861120,879961,148398,SAMD11,NM_152486.2,+
13343,chr1,879582,894679,26155,NOC2L,NM_015658.3,-


In [11]:
gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/gene-20200109.sorted.bed', index=False, sep='\t')

In [12]:
!bgzip -cf data/gene-20200109.sorted.bed > ../data/gene-20200109.sorted.bed.gz

In [13]:
!tabix -fp bed ../data/gene-20200109.sorted.bed.gz

In [14]:
with open('data/omim-gene.list') as f:
    omim_gene = f.read().split('\n')

In [15]:
omim_gene = gene[gene['name2'].isin(omim_gene)].copy()

In [16]:
omim_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
22498,chr1,948846,949920,9636,ISG15,NM_005101.3,+
8184,chr1,955502,991499,375790,AGRN,NM_198576.3,+
20527,chr1,1146705,1149548,7293,TNFRSF4,NM_003327.3,-
20581,chr1,1167628,1170421,126792,B3GALT6,NM_080605.3,+
13882,chr1,1270657,1284492,1855,DVL1,NM_004421.2,-


In [17]:
omim_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/omim-gene-20200109.sorted.bed', index=False, sep='\t')

In [18]:
!bgzip -cf data/omim-gene-20200109.sorted.bed > ../data/omim-gene-20200109.sorted.bed.gz

In [19]:
!tabix -fp bed ../data/omim-gene-20200109.sorted.bed.gz

In [20]:
curation_gene = pd.read_csv('data/ClinGen_gene_curation_list_GRCh37.tsv', sep='\t', dtype=str, skiprows=5)

In [21]:
curation_gene.head()

,#Gene Symbol,Gene ID,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID
0,A4GALT,53947,22q13.2,chr22:43088121-43117307,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,2014-12-11,111400,NaN
1,AAGAB,79719,15q23,chr15:67493013-67547536,3,Sufficient evidence for dosage pathogenicity,23064416,23000146,NaN,0,No evidence available,NaN,NaN,NaN,2013-02-28,148600,NaN
2,AARS,16,16q22.1,chr16:70286297-70323412,0,No evidence available,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,2018-01-11,NaN,NaN
3,AARS2,57505,6p21.1,chr6:44266463-44281063,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,2016-08-22,NaN,NaN
4,AASS,10157,7q31.32,chr7:121713598-121784344,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,2016-08-22,NaN,NaN


In [22]:
hi_genes = set(
    curation_gene.loc[
        curation_gene['Haploinsufficiency Score'] == '3', '#Gene Symbol'
    ]
)

In [23]:
hi_gene = gene[gene['name2'].isin(hi_genes)].copy()

In [24]:
hi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
59,chr1,6845383,7829766,23261,CAMTA1,NM_015215.3,+
8264,chr1,17345216,17380665,6390,SDHB,NM_003000.2,-
3896,chr1,27022521,27108601,8289,ARID1A,NM_139135.2,+
4768,chr1,27860755,27930005,27245,AHDC1,NM_001029882.3,-
8525,chr1,43391045,43424847,6513,SLC2A1,NM_006516.2,-


In [25]:
hi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-gene-20200109.sorted.bed', index=False, sep='\t')

In [26]:
!bgzip -cf data/hi-gene-20200109.sorted.bed > ../data/hi-gene-20200109.sorted.bed.gz

In [27]:
!tabix -fp bed ../data/hi-gene-20200109.sorted.bed.gz

In [28]:
hi_cds = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes), ['chrom', 'cdsStart', 'cdsEnd', 'GeneID', 'name2', 'name']
].copy()
hi_cds = hi_cds.sort_values(['chrom', 'cdsStart', 'cdsEnd'])

In [29]:
hi_cds.head()

,chrom,cdsStart,cdsEnd,GeneID,name2,name
59,chr1,6845590,7826551,23261,CAMTA1,NM_015215.3
8264,chr1,17345375,17380514,6390,SDHB,NM_003000.2
3896,chr1,27022894,27107247,8289,ARID1A,NM_139135.2
4768,chr1,27873814,27878626,27245,AHDC1,NM_001029882.3
8525,chr1,43392711,43424322,6513,SLC2A1,NM_006516.2


In [31]:
hi_cds.rename(columns={
    'chrom': '#chrom', 'cdsStart': 'start', 'cdsEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-cds-20200109.sorted.bed', index=False, sep='\t')

In [32]:
!bgzip -cf data/hi-cds-20200109.sorted.bed > ../data/hi-cds-20200109.sorted.bed.gz

In [33]:
!tabix -fp bed ../data/hi-cds-20200109.sorted.bed.gz

In [34]:
hi_exons = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes), ['chrom', 'exonStarts', 'exonEnds', 'GeneID', 'name2', 'name', 'strand']
].copy()

In [35]:
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
2,chrX,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",1756,DMD,NM_000109.3,-
21,chrX,"28605680,28807436,29301054,29414374,29417271,2...","28606164,28807542,29301334,29414561,29417425,2...",11141,IL1RAPL1,NM_014271.3,+
33,chr7,"69063904,69364271,69583117,69599521,69900737,7...","69064948,69364484,69583219,69599557,69900767,7...",26053,AUTS2,NM_015570.3,+
46,chr2,"50145642,50170850,50280408,50282092,50318460,5...","50149389,50170929,50280728,50282182,50318632,5...",9378,NRXN1,NM_001330094.1,-
55,chr12,"23682437,23689386,23696144,23699249,23716191,2...","23687456,23689603,23696318,23699358,23716337,2...",6660,SOX5,NM_152989.4,-


In [36]:
hi_exons['exonStarts'] = hi_exons['exonStarts'].str.replace(r',$', '')
hi_exons['exonEnds'] = hi_exons['exonEnds'].str.replace(r',$', '')

In [37]:
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
2,chrX,"31137344,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",1756,DMD,NM_000109.3,-
21,chrX,"28605680,28807436,29301054,29414374,29417271,2...","28606164,28807542,29301334,29414561,29417425,2...",11141,IL1RAPL1,NM_014271.3,+
33,chr7,"69063904,69364271,69583117,69599521,69900737,7...","69064948,69364484,69583219,69599557,69900767,7...",26053,AUTS2,NM_015570.3,+
46,chr2,"50145642,50170850,50280408,50282092,50318460,5...","50149389,50170929,50280728,50282182,50318632,5...",9378,NRXN1,NM_001330094.1,-
55,chr12,"23682437,23689386,23696144,23699249,23716191,2...","23687456,23689603,23696318,23699358,23716337,2...",6660,SOX5,NM_152989.4,-


In [38]:
start = hi_exons['exonStarts'].str.split(',').apply(pd.Series).stack().reset_index()
start = start.rename(columns={'level_0': 'row', 0: 'start'})[['row', 'start']]
start['start'] = start['start'].astype(int)
end = hi_exons['exonEnds'].str.split(',').apply(pd.Series).stack().reset_index()
end = end.rename(columns={0: 'end'})['end'].astype(int)
position = start.join(end)

In [39]:
position.head()

,row,start,end
0,2,31137344,31140047
1,2,31144758,31144790
2,2,31152218,31152311
3,2,31164407,31164531
4,2,31165391,31165635


In [40]:
exon = position.merge(
    hi_exons[['chrom', 'GeneID', 'name2', 'name', 'strand']], how='left', left_on='row', right_index=True
)
exon = exon.sort_values(['chrom', 'start', 'end'])

In [41]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand
149,59,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+
150,59,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+
151,59,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+
152,59,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+
153,59,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+


In [42]:
exon['+'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=True).astype(int)
exon['-'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=False).astype(int)

In [43]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
149,59,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23
150,59,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22
151,59,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21
152,59,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20
153,59,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19


In [44]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
1511,1511,154227753,154227875,chrX,2157,F8,NM_000132.3,-,25,2
1512,1511,154250684,154250998,chrX,2157,F8,NM_000132.3,-,26,1
4621,17777,154487518,154490514,chrX,116442,RAB39B,NM_171998.3,-,1,2
4622,17777,154493358,154493874,chrX,116442,RAB39B,NM_171998.3,-,2,1
4685,23010,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1


In [45]:
exon['exon'] = pd.concat([exon.loc[exon['strand'] == '+', '+'], exon.loc[exon['strand'] == '-', '-']])

In [46]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
149,59,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23,1
150,59,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22,2
151,59,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21,3
152,59,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20,4
153,59,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19,5


In [47]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
1511,1511,154227753,154227875,chrX,2157,F8,NM_000132.3,-,25,2,2
1512,1511,154250684,154250998,chrX,2157,F8,NM_000132.3,-,26,1,1
4621,17777,154487518,154490514,chrX,116442,RAB39B,NM_171998.3,-,1,2,2
4622,17777,154493358,154493874,chrX,116442,RAB39B,NM_171998.3,-,2,1,1
4685,23010,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1,1


In [48]:
exon['last_exon'] = exon.groupby(['name2', 'name'])['exon'].transform('max') == exon['exon']

In [49]:
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
149,59,6845383,6845635,chr1,23261,CAMTA1,NM_015215.3,+,1,23,1,False
150,59,6880240,6880310,chr1,23261,CAMTA1,NM_015215.3,+,2,22,2,False
151,59,6885151,6885270,chr1,23261,CAMTA1,NM_015215.3,+,3,21,3,False
152,59,7151363,7151431,chr1,23261,CAMTA1,NM_015215.3,+,4,20,4,False
153,59,7309550,7309686,chr1,23261,CAMTA1,NM_015215.3,+,5,19,5,False


In [50]:
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
1511,1511,154227753,154227875,chrX,2157,F8,NM_000132.3,-,25,2,2,False
1512,1511,154250684,154250998,chrX,2157,F8,NM_000132.3,-,26,1,1,False
4621,17777,154487518,154490514,chrX,116442,RAB39B,NM_171998.3,-,1,2,2,True
4622,17777,154493358,154493874,chrX,116442,RAB39B,NM_171998.3,-,2,1,1,False
4685,23010,2654895,2655782,chrY,6736,SRY,NM_003140.2,-,1,1,1,True


In [51]:
exon = exon[
    ['chrom', 'start', 'end', 'GeneID', 'name2', 'name', 'exon', 'last_exon']
].sort_values(['chrom', 'start', 'end'])

In [52]:
exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
149,chr1,6845383,6845635,23261,CAMTA1,NM_015215.3,1,False
150,chr1,6880240,6880310,23261,CAMTA1,NM_015215.3,2,False
151,chr1,6885151,6885270,23261,CAMTA1,NM_015215.3,3,False
152,chr1,7151363,7151431,23261,CAMTA1,NM_015215.3,4,False
153,chr1,7309550,7309686,23261,CAMTA1,NM_015215.3,5,False


In [53]:
exon.rename(columns={
    'chrom': '#chrom',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/hi-exon-20200109.sorted.bed', index=False, sep='\t')

In [54]:
!bgzip -cf data/hi-exon-20200109.sorted.bed > ../data/hi-exon-20200109.sorted.bed.gz

In [55]:
!tabix -fp bed ../data/hi-exon-20200109.sorted.bed.gz

In [56]:
last_exon = exon[exon['last_exon']]

In [57]:
last_exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
171,chr1,7826518,7829766,23261,CAMTA1,NM_015215.3,23,True
3726,chr1,17345216,17345453,6390,SDHB,NM_003000.2,8,True
2508,chr1,27105513,27108601,8289,ARID1A,NM_139135.2,20,True
2798,chr1,27860755,27861427,27245,AHDC1,NM_001029882.3,7,True
3802,chr1,43391045,43392912,6513,SLC2A1,NM_006516.2,10,True


In [58]:
last_exon_region = last_exon['chrom'] + ':' + last_exon['start'].astype(str) + '-' + last_exon['end'].astype(str)
last_exon_region = last_exon_region.str.replace('chr', '')

In [59]:
last_exon_region.head()

171       1:7826518-7829766
3726    1:17345216-17345453
2508    1:27105513-27108601
2798    1:27860755-27861427
3802    1:43391045-43392912
dtype: object

In [60]:
need_fields = [
    'variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',
    'variants/AF_ESP', 'variants/AF_EXAC', 'variants/AF_TGP', 'variants/CLNSIG'
]

In [61]:
with open('data/clinvar_pathogenic_20191125.sorted.vcf', 'w') as f:
    headers = allel.read_vcf_headers('data/clinvar_20191125.vcf.gz')
    f.write(''.join(headers.headers))
    
    for region in last_exon_region:
        fields, samples, headers, it = allel.iter_vcf_chunks(
            'data/clinvar_20191125.vcf.gz', fields=need_fields, alt_number=1, region=region
        )
        for variants, *_ in it:
            esp_filter = np.isnan(variants['variants/AF_ESP'])
            esp_filter[~esp_filter] |= variants['variants/AF_ESP'][~esp_filter] < 0.01

            exac_filter = np.isnan(variants['variants/AF_EXAC'])
            exac_filter[~exac_filter] |= variants['variants/AF_EXAC'][~exac_filter] < 0.01

            tgp_filter = np.isnan(variants['variants/AF_TGP'])
            tgp_filter[~tgp_filter] |= variants['variants/AF_TGP'][~tgp_filter] < 0.01

            pathogenic_filter = np.isin(
                variants['variants/CLNSIG'], ['Likely_pathogenic', 'Pathogenic', 'Pathogenic/Likely_pathogenic']
            )

            af_filter = esp_filter & exac_filter & tgp_filter & pathogenic_filter

            filtered_variants = {k: v[af_filter] for k, v in variants.items()}

            filtered_variants['variants/CHROM'] = 'chr' + filtered_variants['variants/CHROM']

            names, callset = allel.normalize_callset(filtered_variants)
            allel.write_vcf_data(f, names, callset, None, {field: np.nan for field in need_fields})

In [62]:
!sed -i 's/AF_.\+=nan;//' data/clinvar_pathogenic_20191125.sorted.vcf

sed: 1: "data/clinvar_pathogenic ...": extra characters at the end of d command


In [63]:
!bgzip -cf data/clinvar_pathogenic_20191125.sorted.vcf > ../data/clinvar-pathogenic-20191125.sorted.vcf.gz

In [64]:
!tabix -fp vcf ../data/clinvar-pathogenic-20191125.sorted.vcf.gz

In [65]:
uhi_genes = set(
    curation_gene.loc[curation_gene['Haploinsufficiency Score'] == '40', '#Gene Symbol']
)

In [66]:
uhi_gene = gene[gene['name2'].isin(uhi_genes)].copy()

In [67]:
uhi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
10291,chr1,55505148,55530526,255738,PCSK9,NM_174936.3,+
18375,chr13,20761601,20767114,2706,GJB2,NM_004004.5,-


In [68]:
uhi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/uhi-gene-20200109.sorted.bed', index=False, sep='\t')

In [69]:
!bgzip -cf data/uhi-gene-20200109.sorted.bed > ../data/uhi-gene-20200109.sorted.bed.gz

In [70]:
!tabix -fp bed ../data/uhi-gene-20200109.sorted.bed.gz

In [71]:
region = pd.read_csv('data/ClinGen_region_curation_list_GRCh37.tsv', sep='\t', skiprows=5, dtype=str)

In [72]:
region.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID
0,ISCA-46300,15q24 recurrent region (C-D) (includes SIN3A),NaN,chr15: 75631787-75972909,3,Sufficient evidence for dosage pathogenicity,27399968,22180641,NaN,0,No evidence available,NaN,NaN,NaN,2020-01-13,613406,NaN
1,ISCA-46299,Xp11.22 region (includes HUWE1),Xp11.22,chrX:53363456-53793054,0,No evidence available,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,22840365,20655035,26692240,2018-11-19,NaN,NaN
2,ISCA-46298,13q12.12 recurrent region (includes SACS and S...,13q12.12,chr13:23544582-24881361,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,23239789,NaN,NaN,2019-11-03,"270550,253700",NaN
3,ISCA-46297,16p12.2 recurrent region (includes OTOA) (dist...,16p12.2,chr16:21570113-21740423,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,40,Dosage sensitivity unlikely,NaN,NaN,NaN,2019-11-03,607039,NaN
4,ISCA-46296,15q24 recurrent region (A-C),15q24.1-q24.2,chr15:72963715-75508312,3,Sufficient evidence for dosage pathogenicity,22180641,19921647,NaN,1,Little evidence for dosage pathogenicity,19557438,20860070,NaN,2020-01-03,NaN,NaN


In [73]:
position = region['Genomic Location'].str.extract(r'(?P<chrom>chr\w+)\s*:\s*(?P<start>\d+)\s*-\s*(?P<end>\d+)')
position['start'] = position['start'].astype(int)
position['end'] = position['end'].astype(int)

In [74]:
position.head()

,chrom,start,end
0,chr15,75631787,75972909
1,chrX,53363456,53793054
2,chr13,23544582,24881361
3,chr16,21570113,21740423
4,chr15,72963715,75508312


In [75]:
region_pos = region.merge(position, how='left', left_index=True, right_index=True)

In [76]:
region_pos.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Date Last Evaluated,Loss phenotype OMIM ID,Triplosensitive phenotype OMIM ID,chrom,start,end
0,ISCA-46300,15q24 recurrent region (C-D) (includes SIN3A),NaN,chr15: 75631787-75972909,3,Sufficient evidence for dosage pathogenicity,27399968,22180641,NaN,0,No evidence available,NaN,NaN,NaN,2020-01-13,613406,NaN,chr15,75631787,75972909
1,ISCA-46299,Xp11.22 region (includes HUWE1),Xp11.22,chrX:53363456-53793054,0,No evidence available,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,22840365,20655035,26692240,2018-11-19,NaN,NaN,chrX,53363456,53793054
2,ISCA-46298,13q12.12 recurrent region (includes SACS and S...,13q12.12,chr13:23544582-24881361,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,0,No evidence available,23239789,NaN,NaN,2019-11-03,"270550,253700",NaN,chr13,23544582,24881361
3,ISCA-46297,16p12.2 recurrent region (includes OTOA) (dist...,16p12.2,chr16:21570113-21740423,30,Gene associated with autosomal recessive pheno...,NaN,NaN,NaN,40,Dosage sensitivity unlikely,NaN,NaN,NaN,2019-11-03,607039,NaN,chr16,21570113,21740423
4,ISCA-46296,15q24 recurrent region (A-C),15q24.1-q24.2,chr15:72963715-75508312,3,Sufficient evidence for dosage pathogenicity,22180641,19921647,NaN,1,Little evidence for dosage pathogenicity,19557438,20860070,NaN,2020-01-03,NaN,NaN,chr15,72963715,75508312


In [77]:
hi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [78]:
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
43,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD)
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
17,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3)
51,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (LCR-3/4-flanked...
63,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region


In [79]:
hi_region['omim_genes'] = hi_region.apply(
    lambda row: ','.join(omim_gene.loc[
        (omim_gene['chrom'] == row['chrom']) & (omim_gene['txStart'] >= row['start']) & (omim_gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [80]:
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,omim_genes
43,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD),"ISG15,AGRN,TNFRSF4,B3GALT6,DVL1,ATAD3A,TMEM240..."
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc...","GJA5,GJA8"
17,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3),"SDCCAG8,AKT3,ZBTB18,COX20,HNRNPU"
51,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (LCR-3/4-flanked...,"ANXA11,MAT1A,CDHR1,RGR,LDB3,BMPR1A"
63,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region,"PAX6,WT1"


In [81]:
hi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/hi-region-20200109.sorted.bed', sep='\t', index=False)

In [82]:
!bgzip -cf data/hi-region-20200109.sorted.bed > ../data/hi-region-20200109.sorted.bed.gz

In [83]:
!tabix -fp bed ../data/hi-region-20200109.sorted.bed.gz

In [84]:
uhi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [85]:
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region)
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region)
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere
21,chr16,34202088,35147508,ISCA-37481,3 copies: 16p centromere
20,chr19,43242796,43741310,ISCA-37483,19q13.3 region (PSG gene cluster)


In [86]:
uhi_region['genes'] = uhi_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom']) & (gene['txStart'] >= row['start']) & (gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [87]:
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region),"FAM72C,PPIAL4C,FCGR1A"
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region),"OR4E2,OR4E1"
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere,
21,chr16,34202088,35147508,ISCA-37481,3 copies: 16p centromere,
20,chr19,43242796,43741310,ISCA-37483,19q13.3 region (PSG gene cluster),"PSG8,PSG1,PSG6,PSG7,PSG11,PSG2,PSG5,PSG4"


In [88]:
uhi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/uhi-region-20200109.sorted.bed', sep='\t', index=False)

In [89]:
!bgzip -cf data/uhi-region-20200109.sorted.bed > ../data/uhi-region-20200109.sorted.bed.gz

In [90]:
!tabix -fp bed ../data/uhi-region-20200109.sorted.bed.gz

In [91]:
with open('data/gene-table.json') as f:
    data = json.load(f)
    decipher = pd.DataFrame.from_records(data, columns=['name', 'hi_score', 'p_li']).rename(columns={'name': 'symbol'})
    decipher = decipher.merge(gene, left_on='symbol', right_on='name2')

In [92]:
decipher = decipher.loc[
    (decipher['p_li'] >= 0.9) & (decipher['hi_score'] < 0.1),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'p_li', 'hi_score']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [93]:
decipher.head()

,chrom,txStart,txEnd,GeneID,name2,name,p_li,hi_score
2112,chr1,2439969,2458067,55229,PANK4,NM_018216.2,0.945333,0.098966
2587,chr1,155290639,155300909,23623,RUSC1,NM_001105204.1,0.949429,0.035285
2705,chr1,232533710,232651354,57568,SIPA1L2,NM_020808.4,0.999220,0.079950
1608,chr10,852853,977645,23185,LARP4B,NM_015155.2,0.999886,0.071477
199,chr10,24872537,25012597,57584,ARHGAP21,NM_020824.3,0.999947,0.073197


In [94]:
decipher.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/decipher-gene-20200109.sorted.bed', sep='\t', index=False)

In [95]:
!bgzip -cf data/decipher-gene-20200109.sorted.bed > ../data/decipher-gene-20200109.sorted.bed.gz

In [96]:
!tabix -fp bed ../data/decipher-gene-20200109.sorted.bed.gz

In [97]:
ts_genes = set(
    curation_gene.loc[
        curation_gene['Triplosensitivity Score'] == '3', '#Gene Symbol'
    ]
)

In [98]:
ts_gene = gene[gene['name2'].isin(ts_genes)].copy()

In [99]:
ts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
5423,chr5,126112314,126172712,4001,LMNB1,NR_134488.1,+
12962,chrX,103031433,103047548,5354,PLP1,NM_001128834.2,+
4411,chrX,153287263,153363188,4204,MECP2,NM_004992.3,-


In [100]:
ts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/ts-gene-20200109.sorted.bed', index=False, sep='\t')

In [101]:
!bgzip -cf data/ts-gene-20200109.sorted.bed > ../data/ts-gene-20200109.sorted.bed.gz

In [102]:
!tabix -fp bed ../data/ts-gene-20200109.sorted.bed.gz

In [103]:
uts_genes = set(
    curation_gene.loc[curation_gene['Triplosensitivity Score'] == '40', '#Gene Symbol']
)

In [104]:
uts_gene = gene[gene['name2'].isin(uts_genes)].copy()

In [105]:
uts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
21975,chrX,6451658,6453159,51481,VCX3A,NM_016379.3,-


In [106]:
uts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv('data/uts-gene-20200109.sorted.bed', index=False, sep='\t')

In [107]:
!bgzip -cf data/uts-gene-20200109.sorted.bed > ../data/uts-gene-20200109.sorted.bed.gz

In [108]:
!tabix -fp bed ../data/uts-gene-20200109.sorted.bed.gz

In [109]:
ts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [110]:
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
62,chr15,22832519,28379874,ISCA-37404,"15q11q13 recurrent (PWS/AS) region (BP1-BP3, C..."
23,chr15,23747996,28379874,ISCA-37478,"15q11q13 recurrent (PWS/AS) region (BP2-BP3, C..."
64,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (..."
47,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...


In [111]:
ts_region['omim_genes'] = ts_region.apply(
    lambda row: ','.join(omim_gene.loc[
        (omim_gene['chrom'] == row['chrom']) & (omim_gene['txStart'] >= row['start']) & (omim_gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [112]:
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,omim_genes
52,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc...","GJA5,GJA8"
62,chr15,22832519,28379874,ISCA-37404,"15q11q13 recurrent (PWS/AS) region (BP1-BP3, C...","NIPA1,MKRN3,MAGEL2,NDN,SNRPN,UBE3A,GABRB3,GABR..."
23,chr15,23747996,28379874,ISCA-37478,"15q11q13 recurrent (PWS/AS) region (BP2-BP3, C...","MKRN3,MAGEL2,NDN,SNRPN,UBE3A,GABRB3,GABRA5,OCA2"
64,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (...","KIF22,PRRT2,ALDOA,TBX6"
47,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...,"INPP5K,PRPF8,WDR81,SERPINF2,SERPINF1,DPH1,PAFA..."


In [113]:
ts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/ts-region-20200109.sorted.bed', sep='\t', index=False)

In [114]:
!bgzip -cf data/ts-region-20200109.sorted.bed > ../data/ts-region-20200109.sorted.bed.gz

In [115]:
!tabix -fp bed ../data/ts-region-20200109.sorted.bed.gz

In [116]:
uts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])

In [117]:
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region)
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region)
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere
33,chr15,22832519,23090897,ISCA-37448,15q11.2 recurrent region (BP1-BP2) (includes N...
5,chr15,32019621,32445405,ISCA-46295,15q13.3 recurrent region (D-CHRNA7 to BP5) (in...


In [118]:
uts_region['genes'] = uts_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom']) & (gene['txStart'] >= row['start']) & (gene['txEnd'] <= row['end']), 'name2'
    ]),
    axis=1
)

In [119]:
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
29,chr1,148867551,149768855,ISCA-37469,1q21.2 region (polymorphic region),"FAM72C,PPIAL4C,FCGR1A"
25,chr14,22111109,23021097,ISCA-37476,14q11.2 region (TCRA region),"OR4E2,OR4E1"
24,chr14,106050000,107289540,ISCA-37477,1 copy: 14q telomere; 3 copies: 14q telomere,
33,chr15,22832519,23090897,ISCA-37448,15q11.2 recurrent region (BP1-BP2) (includes N...,"TUBGCP5,CYFIP1,NIPA2,NIPA1"
5,chr15,32019621,32445405,ISCA-46295,15q13.3 recurrent region (D-CHRNA7 to BP5) (in...,


In [120]:
uts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv('data/uts-region-20200109.sorted.bed', sep='\t', index=False)

In [121]:
!bgzip -cf data/uts-region-20200109.sorted.bed > ../data/uts-region-20200109.sorted.bed.gz

In [122]:
!tabix -fp bed ../data/uts-region-20200109.sorted.bed.gz